In [1]:
!pip install swig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 81.8 MB/s eta 0:00:00


In [2]:
!pip install stable_baselines3 gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 29.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 18.3 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp312-cp312-linux_x86_64.whl size=2398998 sha256=959307cfc226c6163272eb02fed9697ca18371b77f2a39c0b4fe2bf92d57ea95
  Stored in directory: /root/.cache/pip/wheels/2a/e9/60/774da0bcd07f7dc7761a8590fa2d065e4069568e78dcdc3318
Successfully built box2d-py


In [3]:
import os, platform, warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="This system does not have CUDA")
warnings.filterwarnings("ignore", message="pkg_resources is deprecated")

import numpy as np
import torch
import matplotlib.pyplot as plt

import gymnasium as gym
from stable_baselines3 import SAC
from stable_baselines3.common.atari_wrappers import WarpFrame  # 84x84 grayscale
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import (
    VecFrameStack, VecTransposeImage, VecNormalize, VecVideoRecorder
)
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.torch_layers import NatureCNN
from importlib.metadata import version

print(f"Python: {platform.python_version()}")
print(f"Torch:  {version('torch')} | CUDA avail: {torch.cuda.is_available()} | torch.cuda: {torch.version.cuda}")
print(f"Gymnasium: {version('gymnasium')}")
print(f"SB3: {version('stable_baselines3')}")
print(f"Numpy: {version('numpy')}  Matplotlib: {version('matplotlib')}")


Python: 3.12.12
Torch:  2.9.0+cu126 | CUDA avail: True | torch.cuda: 12.6
Gymnasium: 1.2.2
SB3: 2.7.0
Numpy: 2.0.2  Matplotlib: 3.10.0


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:
# --------------------
# Config (faster run)
# --------------------
ENV_ID = "CarRacing-v3"
LOG_DIR = f"./logs/{ENV_ID}"
VIDEO_DIR = "./videos"
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

TOTAL_STEPS = 500_000       # ↓ from 1M
EVAL_FREQ = 20_000          # evaluate a bit less often
EVAL_EPISODES = 5           # fewer episodes during training to save time
FINAL_EVAL_EPISODES = 20    # do a fuller eval at the end for your report
SEED = 42

USE_GRAY = True
WRAPPER_CLASS = WarpFrame if USE_GRAY else None

def make_training_env(seed=SEED):
    env = make_vec_env(ENV_ID, n_envs=1, seed=seed, wrapper_class=WRAPPER_CLASS)
    env = VecFrameStack(env, n_stack=4)
    env = VecTransposeImage(env)
    # Reward normalization helps SAC stability on pixels
    env = VecNormalize(env, norm_obs=False, norm_reward=True, clip_reward=10.0)
    return env

def make_eval_env(seed=SEED+1):
    env = make_vec_env(ENV_ID, n_envs=1, seed=seed, wrapper_class=WRAPPER_CLASS)
    env = VecFrameStack(env, n_stack=4)
    env = VecTransposeImage(env)
    env = VecNormalize(env, norm_obs=False, norm_reward=True, clip_reward=10.0, training=False)
    return env

train_env = make_training_env(SEED)
eval_env = make_eval_env(SEED + 1)

policy_kwargs = dict(
    features_extractor_class=NatureCNN,
    features_extractor_kwargs=dict(features_dim=512),  # stronger CNN head
    net_arch=[256, 256],
)

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=LOG_DIR,
    log_path=LOG_DIR,
    eval_freq=EVAL_FREQ,
    n_eval_episodes=EVAL_EPISODES,
    deterministic=True,
    render=False,
)


/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

In [5]:
# --------------------
# SAC hyperparams (slightly lighter/faster)
# --------------------
model = SAC(
    "CnnPolicy",
    train_env,
    learning_rate=1e-4,
    buffer_size=500_000,        # ↓ from 1,000,000 to reduce RAM/time
    batch_size=256,
    learning_starts=5_000,      # ↓ warmup so learning begins earlier
    gamma=0.99,
    tau=0.005,
    ent_coef="auto",
    target_entropy="auto",
    train_freq=(1, "step"),
    gradient_steps=1,
    policy_kwargs=policy_kwargs,
    verbose=1,
    tensorboard_log=LOG_DIR,
    seed=SEED,
)

Using cuda device


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [6]:
# --------------------
# Train
# --------------------
model.learn(total_timesteps=TOTAL_STEPS, callback=eval_callback, progress_bar=True)

# Save final model and normalization stats
final_model_path = os.path.join(LOG_DIR, "sac_car_racing_final_500k")
model.save(final_model_path)
train_env.save(os.path.join(LOG_DIR, "vecnormalize.pkl"))



Logging to ./logs/CarRacing-v3/SAC_1


Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -31.6    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 94       |
|    time_elapsed    | 42       |
|    total_timesteps | 4000     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -35      |
| time/              |          |
|    episodes        | 8        |
|    fps             | 38       |
|    time_elapsed    | 206      |
|    total_timesteps | 8000     |
| train/             |          |
|    actor_loss      | -23.3    |
|    critic_loss     | 0.0994   |
|    ent_coef        | 0.741    |
|    ent_coef_loss   | -1.51    |
|    learning_rate   | 0.0001   |
|    n_updates       | 2999     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_me

Eval num_timesteps=20000, episode_reward=-18.57 +/- 16.33

...

Episode length: 1000.00 +/- 0.00

---------------------------------
| eval/              |          |
|    mean_ep_length  | 1e+03    |
|    mean_reward     | -90.9    |
| time/              |          |
|    total_timesteps | 480000   |
| train/             |          |
|    actor_loss      | -280     |
|    critic_loss     | 1.09e+03 |
|    ent_coef        | 0.135    |
|    ent_coef_loss   | -0.236   |
|    learning_rate   | 0.0001   |
|    n_updates       | 474999   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -75.5    |
| time/              |          |
|    episodes        | 484      |
|    fps             | 18       |
|    time_elapsed    | 25873    |
|    total_timesteps | 483135   |
| train/             |          |
|    actor_loss      | -195     |
|    critic_loss     | 759      |
|    ent_coef        | 0.133    |
|    ent_coef_loss   | 0.277    |
|    learning_rate   | 0.0001   |
|    n_updates

Eval num_timesteps=500000, episode_reward=-81.52 +/- 7.12

Episode length: 1000.00 +/- 0.00

---------------------------------
| eval/              |          |
|    mean_ep_length  | 1e+03    |
|    mean_reward     | -81.5    |
| time/              |          |
|    total_timesteps | 500000   |
| train/             |          |
|    actor_loss      | -177     |
|    critic_loss     | 5.38e+03 |
|    ent_coef        | 0.166    |
|    ent_coef_loss   | 0.0265   |
|    learning_rate   | 0.0001   |
|    n_updates       | 494999   |
---------------------------------


In [7]:
# --------------------
# Helpers to eval with true env rewards
# --------------------
def load_eval_env_with_stats():
    env = make_eval_env(SEED + 100)
    env = VecNormalize.load(os.path.join(LOG_DIR, "vecnormalize.pkl"), env)
    env.training = False
    env.norm_reward = False  # report true env rewards
    return env

In [8]:
# --------------------
# Evaluate FINAL model
# --------------------
eval_env_final = load_eval_env_with_stats()
final_model = SAC.load(final_model_path, env=eval_env_final)
mean_r, std_r = evaluate_policy(final_model, eval_env_final, n_eval_episodes=FINAL_EVAL_EPISODES, deterministic=True)
print(f"[FINAL @500k] Mean reward: {mean_r:.2f} +/- {std_r:.2f}")
eval_env_final.close()


[FINAL @500k] Mean reward: -74.95 +/- 10.34


In [9]:
# --------------------
# Evaluate BEST model
# --------------------
best_model_path = os.path.join(LOG_DIR, "best_model.zip")
if os.path.exists(best_model_path):
    eval_env_best = load_eval_env_with_stats()
    best_model = SAC.load(best_model_path, env=eval_env_best)
    mean_r_b, std_r_b = evaluate_policy(best_model, eval_env_best, n_eval_episodes=FINAL_EVAL_EPISODES, deterministic=True)
    print(f"[BEST  @500k] Mean reward: {mean_r_b:.2f} +/- {std_r_b:.2f}")
    eval_env_best.close()
else:
    print("No best_model.zip found (EvalCallback may not have improved over initial policy).")


[BEST  @500k] Mean reward: 682.02 +/- 268.29


In [10]:
# --------------------
# Record short video (best if available else final)
# --------------------
def record_video(model_path, out_prefix="best_model"):
    env = load_eval_env_with_stats()
    env = VecVideoRecorder(
        env,
        VIDEO_DIR,
        record_video_trigger=lambda step: step == 0,
        video_length=1000,  # CarRacing horizon ~1000 steps
        name_prefix=f"{out_prefix}_car_racing_sac_500k",
    )
    obs = env.reset()
    # Load model once outside the loop
    policy = SAC.load(model_path, env=env)
    for _ in range(1000):
        action, _ = policy.predict(obs, deterministic=True)
        obs, rewards, dones, infos = env.step(action)
        if dones[0]:
            break
    env.close()

if os.path.exists(best_model_path):
    record_video(best_model_path, out_prefix="best_model")
else:
    record_video(final_model_path, out_prefix="final_model")

print(f"Video(s) saved to: {VIDEO_DIR}")

Saving video to /content/videos/best_model_car_racing_sac_500k-step-0-to-step-1000.mp4


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"


Moviepy - Building video /content/videos/best_model_car_racing_sac_500k-step-0-to-step-1000.mp4.
Moviepy - Writing video /content/videos/best_model_car_racing_sac_500k-step-0-to-step-1000.mp4



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Moviepy - Done !
Moviepy - video ready /content/videos/best_model_car_racing_sac_500k-step-0-to-step-1000.mp4
Video(s) saved to: ./videos


In [11]:
# See it
import glob
glob.glob("/content/videos/*.mp4")

# Download to your computer
from google.colab import files
files.download("/content/videos/best_model_car_racing_sac_500k-step-0-to-step-1000.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

import os, shutil
DEST = "/content/drive/MyDrive/CarRacingVideos"
os.makedirs(DEST, exist_ok=True)
shutil.copy("/content/videos/best_model_car_racing_sac_500k-step-0-to-step-1000.mp4",
            f"{DEST}/best_model_car_racing_sac_500k-step-0-to-step-1000.mp4")
print("Copied to:", DEST)


Mounted at /content/drive
Copied to: /content/drive/MyDrive/CarRacingVideos


In [13]:
# --------------------
# Plot evaluation curve (from EvalCallback)
# --------------------
eval_npz = os.path.join(LOG_DIR, "evaluations.npz")
if os.path.exists(eval_npz):
    data = np.load(eval_npz)
    timesteps = data["timesteps"]
    results = data["results"]
    mean_results = results.mean(axis=1)
    std_results = results.std(axis=1)

    plt.figure()
    plt.plot(timesteps, mean_results, linewidth=2)
    plt.fill_between(timesteps, mean_results - std_results, mean_results + std_results, alpha=0.25)
    plt.xlabel("Timesteps")
    plt.ylabel("Eval Mean Reward (true env reward)")
    plt.title(f"SAC on {ENV_ID} (500k steps; mean ± std over {EVAL_EPISODES} eps / eval)")
    plt.grid(True, linestyle="--", alpha=0.3)
    plt.show()
else:
    print("evaluations.npz not found; was EvalCallback attached?")